In [ ]:
import warnings
warnings.filterwarnings("ignore")

import os
import sys
import time
import json
import numpy as np
import pandas as pd
import geopandas as gpd
import pickle as pkl
import networkx as nx
import matplotlib.pyplot as plt

from pprint import pprint

import src
from src.reload import deep_reload

In [ ]:
'''
Importing California SNG
'''

graph = src.graph.graph_from_json('Outputs/sng_combined_directed.json')

In [ ]:
deep_reload(src)

places = [k for k, v in graph._node.items() if v['type'] == 'place']
stations = [k for k, v in graph._node.items() if v['type'] == 'station']

places = src.graph.subgraph(graph, places)
stations = src.graph.subgraph(graph, stations)

In [ ]:
fig, ax = plt.subplots(1, 2, figsize = (10, 4))

for source, node in places._node.items():

    node['log_population'] = np.log(node['population'])

kw = {
    'show_links': False,
    'node_field': 'log_population',
    'scatter': {
        's': 50,
        'ec': 'k',
        # 'fc': 'none',
        'zorder': 4,
        'label': 'Selected Locations',
    },
    'colorbar': {
        'label': 'Log Population [-]',
    },
}

src.figures.plot_graph(places, ax = ax[1], **kw)

kw = {
    'show_links': False,
    'node_field': 'n_dcfc',
    'scatter': {
        's': 50,
        'ec': 'k',
        'zorder': 0,
        'label': 'Stations',
    },
    'colorbar': {
        'label': 'Number of Ports [-]',
    },
}

src.figures.plot_graph(
   stations, ax = ax[0], **kw,
)

kw = {
    'facecolor': 'whitesmoke',
}

_ = [ax.set(**kw) for ax in ax]

kw = {
    'ls': '--',
}

_ = [ax.grid(**kw) for ax in ax]

In [ ]:
'''
Next step is to define vehicle andsupply station objects.
These objects will be used to add supply costs to edges which begin at a supply station.
'''
deep_reload(src)

seed = 1468952
# seed = None
rng = np.random.default_rng(seed)

risk_attitude = (0, .2)
risk_attitude = (.8, 1)
rho = np.linspace(*risk_attitude, 100)

cases = 30

station_kw = {}

station_kw['place'] = {
    'cases': 1,
    'type': 'ac',
    'access': 'private',
    'price': .4 / 3.6e6,
    'setup_time': 0,
    'rng': rng,
}

station_kw['station'] = {
    'reliability': .5,
    'cases': cases,
    'type': 'dc',
    'access': 'public',
    'power': src.routing._network_power,
    'price': .5 / 3.6e6,
    'setup_time': 300,
    'rng': rng,
}

vehicle_kw = {
    'cases': 1,
    'linear_fraction': .8,
    'risk_attitude': (0, 1),
    # 'max_charge_start_soc': .8,
}

vehicle = src.routing.Vehicle(**vehicle_kw)

graph = src.routing.supply_costs(graph, vehicle, station_kw)

In [ ]:
deep_reload(src)

# vehicle_kw['max_charge_start_soc'] = .5
# vehicle = src.routing.Vehicle(**vehicle_kw)

destinations = [k for k, v in graph._node.items() if v['type'] == 'place']
origins = [destinations[0]]

t0 = time.time()

costs, values, paths = src.routing.shortest_paths(
    graph, origins, objective = vehicle, destinations = destinations,
    terminate_at_destinations = True,
)

print(f'Executed in {time.time() - t0:.4f} seconds')

In [ ]:
len(costs)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize = (6, 4))

for source, node in places._node.items():

    node['plot'] = (
        values[source]['total_time'].mean() - 
        values[source]['driving_time'].mean()
    )/ 3600

kw = {
    'show_links': False,
    'node_field': 'plot',
    'scatter': {
        's': 50,
        'ec': 'k',
        'zorder': 0,
        'label': 'Stations',
    },
    'colorbar': {
        'label': 'Total Time [-]',
    },
}

src.figures.plot_graph(
   places, ax = ax, **kw,
)

kw = {
    'facecolor': 'whitesmoke',
}

ax.set(**kw)

kw = {
    'ls': '--',
}

ax.grid(**kw)

In [ ]:
deep_reload(src)

origins = [k for k, v in graph._node.items() if v['type'] == 'place']

t0 = time.time()

costs, values, paths = src.routing.all_pairs_shortest_paths(
    graph, origins, objective = vehicle,
)

print(f'\n\nExecuted in {time.time() - t0:.4f} seconds')

In [ ]:
fig, ax = plt.subplots(1, 1, figsize = (6, 4))

for source, node in places._node.items():

    node['plot'] = (
        values['Portland (State Line)'][source]['total_time'].mean() - 
        values['Portland (State Line)'][source]['driving_time'].mean()
    )/ 3600

kw = {
    'show_links': False,
    'node_field': 'plot',
    'scatter': {
        's': 50,
        'ec': 'k',
        'zorder': 0,
        'label': 'Stations',
    },
    'colorbar': {
        'label': 'Total Time [-]',
    },
}

src.figures.plot_graph(
   places, ax = ax, **kw,
)

kw = {
    'facecolor': 'whitesmoke',
}

ax.set(**kw)

kw = {
    'ls': '--',
}

ax.grid(**kw)

In [ ]:
cities = src.graph.graph_from_json('Outputs/places.json')

pop_adj = sum([v['population'] for v in cities._node.values()]) / cities.number_of_nodes()

weighted = {k: v['population'] / pop_adj for k, v in cities._node.items()}

In [ ]:
src.routing.impedance(values, field = 'total_time') / 3600

In [ ]:
src.routing.impedance(
    values, field = 'total_time',
    origins = weighted,
    destinations = weighted,
) / 3600

In [ ]:
'''
Importing California SNG
'''

graph = src.graph.graph_from_json('Outputs/sng_other_directed.json')

In [ ]:
deep_reload(src)

seed = 1468952
seed = None
rng = np.random.default_rng(seed)

vehicle_param = src.experiments._vehicle_kwargs
station_param = src.experiments._station_kwargs
graphs = [0]

graph_index, vehicle_kw, station_kw = src.experiments.generate_case(
    graphs, vehicle_param, station_param, rng,
)

graph_index, vehicle_kw, station_kw

In [ ]:
deep_reload(src)

t0 = time.time()

costs, values, paths = src.experiments.run_case(
    graph, vehicle_kw, station_kw, method = 'dijkstra'
)

print(f'\n\nExecuted in {time.time() - t0:.4f} seconds')

In [ ]:
src.routing.impedance(
    values, field = 'total_time',
    origins = weighted,
    destinations = weighted,
) / 3600

In [ ]:
src.routing.impedance(
    values, field = 'routing_time',
    # origins = weighted,
    # destinations = weighted,
) / 3600

In [ ]:
src.routing.impedance(
    values, field = 'total_time',
    origins = weighted,
    destinations = weighted,
) / 3600

In [ ]:
src.routing.impedance(
    values, field = 'routing_time',
    origins = weighted,
    destinations = weighted,
) / 3600